In [0]:
%sql

create schema if not exists anirvandecodes.bronze;

In [0]:

source_path = "abfss://data@anirvandecodesdata.dfs.core.windows.net/staging/diagnosis/"
checkpoint_path = "abfss://data@anirvandecodesdata.dfs.core.windows.net/bronze/diagnosis_raw/checkpoint/"
schema_location = "abfss://data@anirvandecodesdata.dfs.core.windows.net/bronze/diagnosis_raw/schema/"

# Autoloader read
df = (spark.readStream
          .format("cloudFiles")
          .option("cloudFiles.format", "csv")
          .option("header", "true")
          .option("inferSchema", "true")
          .option("cloudFiles.maxFilesPerTrigger", 1) # READ ONE FILE AT A TIME
          .option("cloudFiles.schemaLocation", schema_location)  
          .load(source_path)
     )

# Write Bronze table (append)
(
    df.drop("_rescued_data")
      .writeStream
      .format("delta")
      .option("checkpointLocation", checkpoint_path)
      .outputMode("append")
      .trigger(availableNow=True)   # AVAILABLE NOW → ingests all files & stops
      .toTable("anirvandecodes.bronze.diagnosis_raw")
)

In [0]:
%sql

select * from anirvandecodes.bronze.diagnosis_raw